**Instituto de Informática - UFG**

Disciplina: Inteligência Artificial

Prof. Cedric Luiz de Carvalho


Grupo

---


Analista de dados: 
Isabela Felix França

---

Engenheiro de conhecimento: 
Daniel Campos da Silva

---


## Fazendo upload do dados do Kaggle

In [1]:
# Faz upload do arquivo "kaggle.json" com API Token do perfil do usuário
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [10]:
# Testa se upload foi feito com sucesso
!ls

kaggle.json  patient.zip  sample_data


In [3]:
# Move arquivo para a uma pasta e dá permissões ao sistema
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Fazendo download do dataset
! kaggle datasets download -d mitishaagarwal/patient

  0% 0.00/8.30M [00:00<?, ?B/s]
100% 8.30M/8.30M [00:00<00:00, 76.4MB/s]


In [5]:
# Conferindo o download
!ls

kaggle.json  patient.zip  sample_data


## Transferindo os dados para planilha

In [11]:
import zipfile
import pandas as pd
 
# Cria dataframe do pandas a partir do zip
dados_pacientes = pd.read_csv('patient.zip',compression='zip')

# Exibindo o começo do dataframe
dados_pacientes

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,...,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,Unnamed: 83,hospital_death
0,66154,25312,118,68.0,22.730000,0,Caucasian,M,180.3,Floor,...,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,NaN,0
1,114252,59342,81,77.0,27.420000,0,Caucasian,F,160.0,Floor,...,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,NaN,0
2,119783,50777,118,25.0,31.950000,0,Caucasian,F,172.7,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,NaN,0
3,79267,46918,118,81.0,22.640000,1,Caucasian,F,165.1,Operating Room / Recovery,...,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,NaN,0
4,92056,34377,33,19.0,NaN,0,Caucasian,M,188.0,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,91592,78108,30,75.0,23.060250,0,Caucasian,M,177.8,Floor,...,1.0,0.0,0.0,0.0,0.0,1.0,Sepsis,Cardiovascular,NaN,0
91709,66119,13486,121,56.0,47.179671,0,Caucasian,F,183.0,Floor,...,0.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,NaN,0
91710,8981,58179,195,48.0,27.236914,0,Caucasian,M,170.2,Accident & Emergency,...,1.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,NaN,0
91711,33776,120598,66,NaN,23.297481,0,Caucasian,F,154.9,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,NaN,0


## Tratando os dados

In [12]:
# A tabela veio com uma coluna Unnamed:83 que não apresenta valores. Logo de antemão ela será excluída
dados_pacientes = dados_pacientes.drop(["Unnamed: 83"], axis=1)

# Corrigir escrita errada na planilha
dados_pacientes.apache_2_bodysystem = dados_pacientes.apache_2_bodysystem.replace('Undefined diagnoses', 'Undefined Diagnoses')

# Apaga colunas que não serão utilizadas
dados_pacientes = dados_pacientes.drop(['encounter_id', 'patient_id', 'hospital_id', 'icu_id'], axis=1)

# Preenchendo valores nulos com a média do atributo
dados_pacientes = dados_pacientes.fillna(dados_pacientes.mean())

# Convertendo atributos 'object' para inteiros
cols = dados_pacientes.columns[dados_pacientes.dtypes.eq('object')]
dados_pacientes[cols] = dados_pacientes[cols].astype('category')
dados_pacientes[cols] = dados_pacientes[cols].apply(lambda x: x.cat.codes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  # This is added back by InteractiveShellApp.init_path()


In [13]:
# Selecionando 10% da base para o modelo e equilibrando as classes
dados_pacientes_obito = dados_pacientes[dados_pacientes.hospital_death == 1]
dados_pacientes_vivo = dados_pacientes[dados_pacientes.hospital_death == 0]

quantidade = min(dados_pacientes_obito.shape[0], dados_pacientes_vivo.shape[0]) * 0.10

dados_paciente_obito = dados_pacientes_obito.sample(n = int(quantidade))
dados_paciente_vivo = dados_pacientes_vivo.sample(n = int(quantidade))

# Juntando a base de dados
frames = [dados_paciente_obito, dados_paciente_vivo]
dados_modelo  = pd.concat(frames)

dados_modelo

,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
17867,66.0,25.882964,0,5,1,152.00,1,0,5,0.093056,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
10405,55.0,52.879767,0,2,1,175.30,0,0,2,0.000694,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
12895,72.0,19.051281,0,2,0,165.10,0,0,5,0.151389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,6,1
41405,80.0,23.119157,0,2,1,190.50,0,0,5,0.026389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
21819,73.0,27.929688,0,2,0,160.00,0,0,5,0.075000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90081,67.0,28.783498,0,2,0,162.60,2,0,7,2.016667,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,8,0
31074,81.0,32.937566,1,2,0,152.40,2,0,1,0.122917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
71143,50.0,28.311281,0,2,1,177.80,0,0,3,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
32423,82.0,37.197731,0,2,0,160.02,0,0,5,0.080556,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,3,0


## Implementando soluções de redes neurais

In [14]:
# Separando dados de treino e teste
from sklearn.model_selection import train_test_split
Y = dados_modelo['hospital_death']
X = dados_modelo.drop(['hospital_death'], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, random_state=1)

In [23]:
# Criando uma lista de tuplas com a configuração das camadas intermediárias
# Exemplo: (9, 2, 5) refere-se a 9 neurônios na primeira camada, 2 na segunda e 5 na terceira
# Trabalharemos com todas as configurações possíveis de 2 até 10 neurônios e de 1 até 3 camadas
import itertools

configs = []
for i in range(1, 4):
  configs.extend(list(itertools.product(range(2, 10), repeat=i)))

In [24]:
# Criando o modelo para cada configuração e salvando sua pontuação
from sklearn.neural_network import MLPClassifier
resultados = []
for i in configs:
  clf = MLPClassifier(solver='adam', learning_rate = 'adaptive', activation = "logistic", alpha=1e-5, hidden_layer_sizes=i, random_state=1, max_iter=2000)

  # Treinando o modelo
  clf.fit(X_train, Y_train)

  # Testando o modelo
  pontos = clf.score(X_test, Y_test)

  # Salvando a pontuação do modelo com sua configuração
  resultados.append((pontos, i))

In [25]:
resultados.sort()
resultados

[(0.49747474747474746, (6, 4, 4)),
 (0.5, (2, 2)),
 (0.5, (2, 2, 2)),
 (0.5, (2, 2, 3)),
 (0.5, (2, 2, 7)),
 (0.5, (2, 2, 9)),
 (0.5, (2, 3, 2)),
 (0.5, (2, 3, 3)),
 (0.5, (2, 3, 4)),
 (0.5, (2, 3, 5)),
 (0.5, (2, 3, 7)),
 (0.5, (2, 4, 2)),
 (0.5, (2, 4, 3)),
 (0.5, (2, 4, 8)),
 (0.5, (2, 5, 2)),
 (0.5, (2, 5, 5)),
 (0.5, (2, 5, 8)),
 (0.5, (2, 5, 9)),
 (0.5, (2, 6)),
 (0.5, (2, 6, 2)),
 (0.5, (2, 6, 3)),
 (0.5, (2, 6, 4)),
 (0.5, (2, 6, 6)),
 (0.5, (2, 6, 7)),
 (0.5, (2, 7, 4)),
 (0.5, (2, 7, 5)),
 (0.5, (2, 7, 6)),
 (0.5, (2, 7, 8)),
 (0.5, (2, 7, 9)),
 (0.5, (2, 8, 4)),
 (0.5, (2, 8, 5)),
 (0.5, (2, 8, 7)),
 (0.5, (2, 8, 9)),
 (0.5, (2, 9, 2)),
 (0.5, (2, 9, 5)),
 (0.5, (2, 9, 9)),
 (0.5, (3, 2, 4)),
 (0.5, (3, 2, 6)),
 (0.5, (3, 2, 8)),
 (0.5, (3, 3)),
 (0.5, (3, 3, 2)),
 (0.5, (3, 3, 3)),
 (0.5, (3, 3, 4)),
 (0.5, (3, 3, 5)),
 (0.5, (3, 3, 6)),
 (0.5, (3, 3, 7)),
 (0.5, (3, 3, 8)),
 (0.5, (3, 3, 9)),
 (0.5, (3, 4, 2)),
 (0.5, (3, 4, 3)),
 (0.5, (3, 4, 6)),
 (0.5, (3, 4, 8)),
 (0.5

In [26]:
# Testando para apenas 2 camadas, de até 30 neurônios cada
resultados2 = []
for i in range(1, 31):
  for j in range(1, 31):
    clf = MLPClassifier(solver='adam', learning_rate = 'adaptive', activation = "logistic", alpha=1e-5, hidden_layer_sizes=(i, j), random_state=1, max_iter=2000)

    # Treinando o modelo
    clf.fit(X_train, Y_train)

    # Testando o modelo
    pontos = clf.score(X_test, Y_test)

    # Salvando a pontuação do modelo com sua configuração
    resultados2.append((pontos, (i, j)))

In [28]:
resultados2.sort()
resultados2

[(0.5, (1, 1)),
 (0.5, (1, 2)),
 (0.5, (1, 3)),
 (0.5, (1, 4)),
 (0.5, (1, 5)),
 (0.5, (1, 6)),
 (0.5, (1, 7)),
 (0.5, (1, 8)),
 (0.5, (1, 9)),
 (0.5, (1, 10)),
 (0.5, (1, 11)),
 (0.5, (1, 12)),
 (0.5, (1, 13)),
 (0.5, (1, 14)),
 (0.5, (1, 15)),
 (0.5, (1, 16)),
 (0.5, (1, 17)),
 (0.5, (1, 18)),
 (0.5, (1, 19)),
 (0.5, (1, 20)),
 (0.5, (1, 21)),
 (0.5, (1, 22)),
 (0.5, (1, 23)),
 (0.5, (1, 24)),
 (0.5, (1, 25)),
 (0.5, (1, 26)),
 (0.5, (1, 27)),
 (0.5, (1, 28)),
 (0.5, (1, 29)),
 (0.5, (1, 30)),
 (0.5, (2, 2)),
 (0.5, (2, 6)),
 (0.5, (2, 16)),
 (0.5, (2, 18)),
 (0.5, (2, 30)),
 (0.5, (3, 3)),
 (0.5, (3, 5)),
 (0.5, (3, 19)),
 (0.5, (3, 22)),
 (0.5, (4, 5)),
 (0.5, (5, 1)),
 (0.5, (6, 5)),
 (0.5075757575757576, (8, 3)),
 (0.5176767676767676, (8, 6)),
 (0.5909090909090909, (2, 4)),
 (0.5984848484848485, (2, 14)),
 (0.6111111111111112, (2, 5)),
 (0.6136363636363636, (2, 29)),
 (0.6212121212121212, (2, 9)),
 (0.6262626262626263, (2, 22)),
 (0.6313131313131313, (2, 17)),
 (0.633838383838383

In [29]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', learning_rate = 'adaptive', activation = "logistic", alpha=1e-5, hidden_layer_sizes=(5, 3, 5), random_state=1, max_iter=2000)

# Treinando o modelo
clf.fit(X_train, Y_train)

# Testando o modelo
clf.score(X_test, Y_test)

0.7954545454545454

In [30]:
# Matriz de confusão para (5, 3, 5)
from sklearn.metrics import confusion_matrix
confusion_matrix(Y, clf.predict(X))

array([[646, 145],
       [165, 626]])

In [31]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', learning_rate = 'adaptive', activation = "logistic", alpha=1e-5, hidden_layer_sizes=(13, 15), random_state=1, max_iter=2000)

# Treinando o modelo
clf.fit(X_train, Y_train)

# Testando o modelo
clf.score(X_test, Y_test)

0.7929292929292929

In [32]:
# Matriz de confusão para (13, 15)
from sklearn.metrics import confusion_matrix
confusion_matrix(Y, clf.predict(X))

array([[634, 157],
       [144, 647]])

Por limitações do hardware, não podemos estender tanto a quantidade de camadas e neurônios. Como não percebemos uma grande melhora de performance aumentando de 2 para 3 camadas, preferimos testar mais intensamente somente com 2 camadas. Testando todas as combinações de até 30 neurônios para cada uma das 2 camadas, chegamos a 80% de acurácia para a configuração (13, 15) como nosso melhor resultado. Enquanto uma solução quase tão boa quanto, com 79% de acurácia, mas com muito menos neurônios, é (5, 3, 5). 

Ao avaliar a matriz de confusão, percebemos que não há grandes diferenças entre falsos negativos e positivos, bem como de verdadeiros positivos para com verdadeiros negativos.

A melhora da performance em relação à quantidade de neurônios por camada não aparenta um comportamente bem definido. Entretanto, é notável que quanto menor a quantidade de neurônios, principalmente na primeira camada, pior é a acurácia do modelo.